### Install

In [152]:
#pip install ucimlrepo

In [153]:
#pip install psycopg2

### Imports

In [154]:
# Imports
from ucimlrepo import fetch_ucirepo
import pandas as pd
import psycopg2
from collections import defaultdict
from scipy.stats import entropy
from scipy import stats as kl
from decimal import Decimal

In [155]:
connection = psycopg2.connect(database="Project645", user="vaishnavishah", password="10jan2001", host="localhost", port=5432)

#cursor = connection.cursor()


In [156]:
# cursor.execute("SELECT * from unmarried;")
# record = cursor.fetchall()
# unmarried_df = pd.DataFrame(record, columns=col_names)

In [157]:
# cursor.execute("SELECT * from married;")
# record = cursor.fetchall()
# married_df = pd.DataFrame(record, columns=col_names)

In [158]:
results_kl = defaultdict(list)

In [159]:
views = []
aggregate = ["COUNT", "AVG", "SUM", "MIN", "MAX"]  
discrete = ["age", "workclass"]  
continuous = ["hours_per_week", "capital_gain"]  

k=10
num_partitions=20
delta= 0.01

In [160]:
for f in aggregate:
    for a in discrete:
        for m in continuous:
            views.append((a, m, f))

In [161]:
def get_all_views():
    views=[]
    for f in aggregate:
        for a in discrete:
            for m in continuous:
                views.append((a,m,f))
    return views

def get_query_result( conn, query ) :
    cur = conn.cursor()
    cur.execute( query )
    return cur

def normalize(f_list_1,f_list_2):
    sum_1=sum(f_list_1)
    sum_2=sum(f_list_2)
    norm_1 = [i/sum_1 for i in f_list_1]
    norm_2=[i/sum_2 for i in f_list_2]
    return norm_1,norm_2

def get_utility(f_list_1,f_list_2):
    nf_1,nf_2=normalize(f_list_1,f_list_2)
    nf_1=[float(x) for x in nf_1]
    nf_2=[float(x) for x in nf_2]
    return kl.entropy(nf_1,nf_2)


def get_top_k(dict_kl, k):
    return sorted(dict_kl.items(), key= lambda item: item[1], reverse=True)[:k]

def get_iter(d):
    if isinstance(d,dict):
        return d.items()
    else: #cursor
        return d.fetchall()

def get_agg_lists(o_1,o_2):
    dict_res = defaultdict(list)
    f_list_1 = []
    f_list_2 = []
    for k,v in get_iter(o_1):
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))
    for k,v in get_iter(o_2):
        if k not in dict_res:
            dict_res[k].append(Decimal(1e-10))
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))
    for k,v in dict_res.items():
        if len(v) != 2:
            dict_res[k].append(Decimal(1e-10))
    for i in dict_res.values():
        f_list_1.append(i[0])
        f_list_2.append(i[1])
    return f_list_1,f_list_2


In [162]:
dict_kl=defaultdict(list)
views=get_all_views()
for a,m,f in views:
    #print (a,f,m)
    q1='select '+a+', '+f+'('+m+') from married where '+a+' is not null group by '+a+' order by '+a+';'
    q2='select '+a+', '+f+'('+m+') from unmarried where '+a+' is not null group by '+a+' order by '+a+';'
    #print(q1, q2)
    res_1=get_query_result(connection, q1)
    res_2=get_query_result(connection, q2)
    f_list_1,f_list_2 = get_agg_lists(res_1,res_2)
    dict_kl[a,m,f].append(get_utility(f_list_1,f_list_2))

get_top_k(dict_kl, k)

[(('age', 'capital_gain', 'AVG'), [6.238480995536909]),
 (('age', 'hours_per_week', 'MIN'), [3.238200817020985]),
 (('age', 'capital_gain', 'MAX'), [1.8147251653770289]),
 (('age', 'capital_gain', 'SUM'), [0.5765073642086408]),
 (('age', 'hours_per_week', 'AVG'), [0.565223313172089]),
 (('age', 'hours_per_week', 'COUNT'), [0.399257399242306]),
 (('age', 'capital_gain', 'COUNT'), [0.399257399242306]),
 (('age', 'hours_per_week', 'SUM'), [0.3324075749358738]),
 (('age', 'hours_per_week', 'MAX'), [0.29199732493276814]),
 (('workclass', 'hours_per_week', 'MIN'), [0.15799483973604933])]